# Arbitrage Detection in the Foreign Exchange Market

In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

The foreign exchange market includes the trading of currencies. It is one of the markets with largest trading volume. Given two currencies at any particular time, say the US dollar and the euro, there are two exchange rates between them: one dollar will buy $r_1$ euros, and one euro will buy $r_2$ dollars. It is evident that an arbitrage opportunity would arise if $r_1 r_2 > 1$ since one could simultaneously convert 1 dollar into $r_1$ euros and the $r_1$ euros into $r_1 r_2 > 1$ dollars. These two transactions would net $r_1 r_2 − 1$ dollars without any risk.

An interesting related question is: Can one detect a similar type of arbitrage opportunity involving more than two currencies? In particular, consider the following hypothetical exchange rates among the currencies USD (US Dollars), EUR (Euros), GBP (British Pounds), AUD (Australian Dollars), and JPY (Japanese Yen).

|     | USD     | EUR     | GBP     | AUD     | JPY     |
| --- | ------- | ------- | ------- | ------- | ------- |
| USD | 1       | 0.639   | 0.537   | 1.0835  | 98.89   |
| EUR | 1.564   | 1       | 0.843   | 1.6958  | 154.773 |
| GBP | 1.856   | 1.186   | 1       | 2.014   | 184.122 |
| AUD | 0.9223  | 0.589   | 0.496   | 1       | 91.263  |
| JPY | 0.01011 | 0.00645 | 0.00543 | 0.01095 | 1       |

A simple veriﬁcation shows that there are no arbitrage opportunities involving only two currencies. However, could there be one involving more than two currencies? Could you simply eyeball such an opportunity? If you cannot, can you prove that such an opportunity does not exist?

**Solution**
For convenience, use $i = 1,..., 5$ to index the above ﬁve currencies USD, EUR, GBP, AUD, and JPY in that order. We let $a_{ij}$ denote the exchange rate from currency $i$ to currency $j$. For instance $a_{34} = 2.014$ and $a_{25} = 154.773$. To model a set of transactions with potential for arbitrage, consider the following decision variables:

**Decision variables**
* $x_{ij}$: amount of currency $i$ converted to currency $j$.
* $y_k$: net amount of currency $k$ after all transactions.

**Objective**
$$
\max \sum_{i = 1}^5 y_i
$$

**Constraints**
$$
y_k = \sum_{i = 1}^5 a_{ik}x_{ik} - \sum_{j = 1}^5 x_{kj}
$$

$$
y_k \geq 0, x_{ij} \geq 0
$$

If there exists arbitrage opportunities, this problem will be unbounded. We can add the constraint
$$
y \leq 1
$$

In [2]:
data = pd.read_csv('Arbitrage.csv')
data

,Unnamed: 0,USD,EUR,GBP,AUD,JPY
0,USD,1.00000,0.63900,0.53700,1.08350,98.890
1,EUR,1.56400,1.00000,0.84300,1.69580,154.773
2,GBP,1.85600,1.18600,1.00000,2.01400,184.122
3,AUD,0.92230,0.58900,0.49600,1.00000,91.263
4,JPY,0.01011,0.00645,0.00543,0.01095,1.000


In [3]:
Currency_rate = data.iloc[0:5,1:6].to_numpy()

In [4]:
def arbitrage(C_rate):
    m = ro.Model("Arbitrage")

    N = len(C_rate)

    x = m.dvar( (N,N) ) 
    y = m.dvar( N )

    m.max( y.sum() )

    for k in range(N):
        m.st( y[k] == sum(C_rate[i,k]*x[i,k] for i in range(N)) - sum(x[k,j] for j in range(N)) )
    m.st( x >= 0, 
         y >= 0, 
         y <= 1 )

    m.solve(grb)

#     print( y.get() )

    if any( (True for i in y.get() if i == 1) ):
        print( "-------------------------------")
        print( "There are arbitrage opportunities!" )
    else:
        print( "-------------------------------")
        print( "NO arbitrage opportunity!" )

In [5]:
arbitrage( Currency_rate )

Using license file C:\Users\qinshen.tang\gurobi.lic
Academic license - for non-commercial use only
Being solved by Gurobi...
Solution status: 2
Running time: 0.0000s
-------------------------------
There are arbitrage opportunities!


In [6]:
from forex_python.converter import CurrencyRates                    

Currency = ['USD', 'EUR', 'SGD','CNY','JPY','AUD','INR','GBP','NZD','HKD', 'MYR']
CR_fun = CurrencyRates()   
N = len(Currency)
CR = np.zeros( (N,N) )

for n in range(N):
    for m in range(N):
        CR[n,m] = CR_fun.get_rate(Currency[n], Currency[m])
        
print( CR )

[[1.00000000e+00 9.17852226e-01 1.36126664e+00 6.31904543e+00
  1.15236347e+02 1.35392382e+00 7.69274897e+01 7.58375402e-01
  1.45580542e+00 7.81587884e+00 4.16943552e+00]
 [1.08950000e+00 1.00000000e+00 1.48310000e+00 6.88460000e+00
  1.25550000e+02 1.47510000e+00 8.38125000e+01 8.26250000e-01
  1.58610000e+00 8.51540000e+00 4.54260000e+00]
 [7.34609939e-01 6.74263367e-01 1.00000000e+00 4.64203358e+00
  8.46537658e+01 9.94605893e-01 5.65116985e+01 5.57110107e-01
  1.06944913e+00 5.74162228e+00 3.06290877e+00]
 [1.58251750e-01 1.45251721e-01 2.15422828e-01 1.00000000e+00
  1.82363536e+01 2.14260814e-01 1.21739099e+01 1.20014235e-01
  2.30383755e-01 1.23687651e+00 6.59820469e-01]
 [8.67781760e-03 7.96495420e-03 1.18128236e-02 5.48355237e-02
  1.00000000e+00 1.17491039e-02 6.67562724e-01 6.58104341e-03
  1.26332139e-02 6.78247710e-02 3.61816010e-02]
 [7.38593994e-01 6.77920141e-01 1.00542336e+00 4.66720900e+00
  8.51128737e+01 1.00000000e+00 5.68181818e+01 5.60131517e-01
  1.07524914e+00

In [7]:
arbitrage(CR)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0000s
-------------------------------
NO arbitrage opportunity!


## Currency exchange

Suppose that there are $ N $ available currencies, and assume that one unit of currency $ i $ can be exchanged for $ r_{ij} $ units of currency $ j $. (Naturally, we assume that $ r_{ij} > 0 $.) There are also certain regulations that impose a limit $ u_i $ on the total amount of currency $ i $ that can be exchanged on any given day. Suppose that we start with $ B $ units of currency $ 1 $ and that we would like to maximize the number of units of currency $ N $ that we end up with at the end of the day, through a sequence of currency transactions. Provide a linear programming formulation of this problem. Assume that for any sequence $ i_1,\cdots,i_k $ of currencies, we have $ r_{i_1i_2}r_{i_2i_3}\dots r_{i_{k-1}i_k}r_{i_ki_1} \leq 1 $, which means that wealth cannot be multiplied by going through a cycle of currencies.

We introduce decision variables $ x_{ij} $ for each combination of $ i = 1,2,\cdots,N $ and $ j = 1,2,\cdots,N $ to denote the amount of currency $i$ exchanged for currency $j$.
		
The first class of constraints follows from the fact that there are no profitable cycles of currencies. Therefore, there is no reason to exchange currency from $N$ to any other currency, or exchange from other currency to $1$. Therefore, we can set $ x_{Ni} = 0, i = 1,2,\cdots,N $ and $ x_{i1} = 0, i = 1,2,\cdots,N $.
		
The second class of constraints models that for each currency $i$, the amount that can be exchanged is limited by $u_i$, hence
		$$
		\sum_{j=1}^N x_{ij} \leq u_i, \quad i = 1,2,\cdots,N.
		$$
		
The third class of constraints models that we cannot exchange more currency from $i$ to other currencies than we exchanged to $i$ before. We have
		$$
		\sum_{j=1}^N r_{ji}x_{ji} \geq \sum_{k=1}^N x_{ik}, \quad i = 2,\cdots,N.
		$$
		
The forth constraint models that the amount of currency we can exchange from $1$ to other currencies is naturally upper bounded by $B$
		$$
		\sum_{j=1}^N x_{1j} \leq B
		$$
		
Finally, the amount of currency $N$ is given by
		$$
		\sum_{j=1}^N r_{jN}x_{jN},
		$$
which is the objective we wish to maximize.
		
In summary, we get the following linear programming for this problem
\begin{align}
	\max \quad & \sum_{j=1}^N r_{jN}x_{jN} \\
	\text{s.t.} \quad
	& \sum_{j=1}^N x_{ij} \leq u_i, \quad & i = 1,2,\cdots,N \\
	& \sum_{j=1}^N r_{ji}x_{ji} \geq \sum_{k=1}^N x_{ik}, \quad & i = 2,\cdots,N \\
	& \sum_{j=1}^N x_{1j} \leq B \\
	& x_{Ni} = 0,  x_{i1} = 0, \quad & i = 1,2,\cdots,N  \\
	& x_{ij} \geq 0, \quad & i = 1,2,\cdots,N, j = 1,2,\cdots,N \\
\end{align}

In [8]:
def Currency_exchange( C_rate, B, u ):
    m = ro.Model("Currency Exchange")

    N = len( C_rate )
    x = m.dvar( (N,N) ) 

    m.max( C_rate[:,N - 1] @ x[:, N - 1] )
    
    m.st( x.sum(axis = 1) <= u )
    
    m.st( sum(C_rate[j,i]*x[j,i] for j in range(N)) >= sum(x[i,k] for k in range(N)) for i in range(1,N) )
    
    m.st( x[0,:].sum() <= B )
    
    m.st( x[N-1,:] == 0,
        x[:,0] == 0,
        x >= 0)

    m.solve(grb)

#     print( x.get() )
    
    print( "-------------------------------")
    print( "You will have", m.get(), "currency N!" )

In [10]:
B = 100000
N = len( CR )
np.random.seed(3)
u = np.random.uniform( 5000, 15000, N)
Currency_exchange( CR, B, u )

Being solved by Gurobi...
Solution status: 2
Running time: 0.0000s
-------------------------------
You will have 43812.341002618334 currency N!


In [56]:
B*CR[0,N-1]

418547.24964739074